In [29]:
T = 2; N = 1000;

In [30]:

	rand('seed',1);
	M=ceil(0.1*N);         % number of synapses per neuron
	D=20;                  % maximal conduction delay 
	% excitatory neurons   % inhibitory neurons      % total number 
	Ne=ceil(0.8*N);                Ni=N-Ne; 
	a=[0.02*ones(Ne,1);    0.1*ones(Ni,1)];
	d=[   8*ones(Ne,1);    2*ones(Ni,1)];
	sm=10;                 % maximal synaptic strength

	exw = 6;
	inw = 5;
	
	%Generate connectivity and weights matrix
	conn = zeros(N,N);
	weight = zeros(N,N);
	delaysmat = zeros(N,N);

	% post=ceil([N*rand(Ne,M);Ne*rand(Ni,M)]); 
	% Take special care not to have multiple connections between neurons
	delays = cell(N,D);
	for i=1:Ne
			p=randperm(N);
			post(i,:)=p(1:M);
			conn(i,p(1:M)) = 1;
			conn(i,p(1:M)) = exw;
			for j=1:M
					delay = ceil(D*rand);
					delays{i, delay}(end+1) = j;  % Assign random exc delays
					delaysmat(i,p(j)) = delay;
			end;
	end;
	for i=Ne+1:N
			p=randperm(Ne);
			post(i,:)=p(1:M);
			conn(i,p(1:M)) = 1;
			conn(i,p(1:M)) = -inw;
			delays{i,1}=1:M;                    % all inh delays are 1 ms.
			delaysmat(i,p(1:M)) = 1;
	end;
	
	s=[exw*ones(Ne,M);-inw*ones(Ni,M)];         % synaptic weights
	%sd=zeros(N,M);                          % their derivatives
	
	%Only needed for STDP
	% Make links at postsynaptic targets to the presynaptic weights
	%pre = cell(N,1);
	%aux = cell(N,1);
	%for i=1:Ne
	%		for j=1:D
	%				for k=1:length(delays{i,j})
	%						pre{post(i, delays{i, j}(k))}(end+1) = N*(delays{i, j}(k)-1)+i;
	%						aux{post(i, delays{i, j}(k))}(end+1) = N*(D-1-j)+i; % takes into account delay
	%				end;
	%		end;
	%end;
	
	times = cell(N,1);

	%STDP = zeros(N,1001+D);
	v = -65*ones(N,1);                      % initial values
	u = 0.2.*v;                             % initial values
	firings=[-D 0];                         % spike timings
	

In [32]:
	for sec=1:T                      		% simulation of T seconds
		for t=1:1000                          % simulation of 1 sec
			I=zeros(N,1);
			I(ceil(N*rand))=20;                 % random thalamic input 
			fired = find(v>=30);                % indices of fired neurons
			if length(fired) > 0 
				for i = fired'
					times{i}(end+1) = (sec-1)+t/1000;
				end
			end
			v(fired)=-65;  
			u(fired)=u(fired)+d(fired);
			%STDP(fired,t+D)=0.1;
			%for k=1:length(fired)
			%	sd(pre{fired(k)})=sd(pre{fired(k)})+STDP(N*t+aux{fired(k)});
			%end;
			firings=[firings;t*ones(length(fired),1),fired];
			k=size(firings,1);
			while firings(k,1)>t-D
				del=delays{firings(k,2),t-firings(k,1)+1};
				ind = post(firings(k,2),del);
				I(ind)=I(ind)+s(firings(k,2), del)';
				%sd(firings(k,2),del)=sd(firings(k,2),del)-1.2*STDP(ind,t+D)';
				k=k-1;
			end;
			v=v+0.5*((0.04*v+5).*v+140-u+I);    % for numerical 
			v=v+0.5*((0.04*v+5).*v+140-u+I);    % stability time 
			u=u+a.*(0.2*v-u);                   % step is 0.5 ms
			%STDP(:,t+D+1)=0.95*STDP(:,t+D);     % tau = 20 ms
		end;
		%plot(firings(:,1),firings(:,2),'.');
		%axis([0 1000 0 N]); drawnow;
		%STDP(:,1:D+1)=STDP(:,1001:1001+D);
		ind = find(firings(:,1) > 1001-D);
		firings=[-D 0;firings(ind,1)-1000,firings(ind,2)];

		%Comment out to ignore STDP
		%s(1:Ne,:)=max(0,min(sm,0.01+s(1:Ne,:)+sd(1:Ne,:)));

		%sd=0.9*sd;
	end;

In [33]:
	spikes.times = times;		%Spike times for each unit
	spikes.units = '1'; 		%Units in seconds
	spikes.conn = conn;			%Connectivity matrix
	spikes.weights = weight;	%Connection weight (negative for inhibitory)
	spikes.delays = delaysmat;	%Delay matrix (if applicable)
	spikes.N = N;				%Number of units
	spikes.T = T;

In [35]:
imagesc(delays)

Invalid datatype for Image CData. Numeric or logical matrix required for image CData.


In [36]:
size(delays)

ans =

        1000          20


In [46]:
d=delays(1:1,:)

d = 

  Columns 1 through 4

    [1x2 double]    [1x3 double]    [1x2 double]    [1x7 double]

  Columns 5 through 8

    [1x3 double]    [1x11 double]    [1x6 double]    [1x4 double]

  Columns 9 through 12

    [1x3 double]    [1x7 double]    [1x4 double]    [1x4 double]

  Columns 13 through 16

    [1x4 double]    [1x6 double]    [1x6 double]    [1x4 double]

  Columns 17 through 20

    [1x8 double]    [1x5 double]    [1x6 double]    [1x5 double]


In [43]:
d

d = 

    [1x2 double]


In [44]:
d{1}

ans =

    45    76
